# 동은님 코드

In [ ]:
pip install dash-extensions

In [4]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Create dummy movie data
data = {
    'Movie': ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E'],
    'Distributor': ['Distributor X', 'Distributor Y', 'Distributor X', 'Distributor Z', 'Distributor Y'],
    'Rating': [8.5, 7.9, 6.8, 9.2, 8.0]
}
df = pd.DataFrame(data)

# Create Dash application
app = dash.Dash(__name__)

# Save HTML to file function
def save_graph_to_html(fig, filename='graph.html'):
    fig.write_html(filename)

# Layout configuration
app.layout = html.Div([
    html.H1("Performance of Movie Distributors"),
    dcc.Checklist(
        id='distributor-selector',
        options=[{'label': distributor, 'value': distributor} for distributor in df['Distributor'].unique()],
        value=[],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id='rating-graph'),
    html.Button('Export to HTML', id='export-button', n_clicks=0)
])

# Callback function definition
@app.callback(
    Output('rating-graph', 'figure'),
    [Input('distributor-selector', 'value')]
)
def update_graph(selected_distributors):
    if not selected_distributors:
        return px.scatter()
    else:
        filtered_df = df[df['Distributor'].isin(selected_distributors)]
        fig = px.bar(filtered_df, x='Movie', y='Rating', color='Distributor', barmode='group', title='Movie Performance')
        return fig

@app.callback(
    Output('export-button', 'children'),
    [Input('export-button', 'n_clicks'),
     Input('rating-graph', 'figure')]
)
def download_html(n_clicks, graph_figure):
    if n_clicks > 0 and graph_figure:
        fig = px.bar(df, x='Movie', y='Rating', color='Distributor')
        save_graph_to_html(fig)
        return 'Exported! Click Again to Export Again'
    return 'Export to HTML'

# Run application
if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import plotly.graph_objs as go
import pandas as pd

# 가상의 영화 데이터 생성
data = {
    'Movie': ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E'],
    'Distributor': ['Distributor X', 'Distributor Y', 'Distributor X', 'Distributor Z', 'Distributor Y'],
    'Rating': [8.5, 7.9, 6.8, 9.2, 8.0]
}
df = pd.DataFrame(data)

# 영화 배급사를 기준으로 데이터를 그룹화하여 그래프를 생성하는 함수
def get_figure(distributor):
    filtered_df = df[df['Distributor'] == distributor]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=filtered_df['Movie'], y=filtered_df['Rating'], name=distributor))
    fig.update_layout(title=f'{distributor}의 영화 성적', xaxis_title='영화', yaxis_title='평점')
    return fig

# 기본 그래프 생성
fig = get_figure('Distributor X')

# 각 배급사를 선택할 수 있는 버튼 추가
buttons = [
    dict(
        label=distributor,
        method='update',
        args=[{'x': [df[df['Distributor'] == distributor]['Movie']],
               'y': [df[df['Distributor'] == distributor]['Rating']],
               'type': 'bar'},
              {'title': f'{distributor}의 영화 성적'}]
    )
    for distributor in df['Distributor'].unique()
]

# 레이아웃에 버튼 추가
fig.update_layout(
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
    }],
    title='배급사별 영화 성적',
    xaxis_title='영화',
    yaxis_title='평점'
)

# 그래프를 HTML로 내보내기
fig.write_html('interactive_movie_ratings.html')

print("인터랙티브 그래프가 interactive_movie_ratings.html로 저장되었습니다.")


인터랙티브 그래프가 interactive_movie_ratings.html로 저장되었습니다.


# 현준님 코드

In [9]:
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv('../5.2_dataset_forviz_add_gener.csv')

# Convert sales column to numeric
df['매출액'] = pd.to_numeric(df['매출액'].str.replace(',', ''), errors='coerce')

# Drop rows with missing sales data
df = df.dropna(subset=['매출액'])

# Convert audience column to numeric
df['관객수'] = pd.to_numeric(df['관객수'].str.replace(',', ''), errors='coerce')

# Drop rows with missing audience data
df = df.dropna(subset=['관객수'])

# Convert release date to datetime
df['개봉일'] = pd.to_datetime(df['개봉일'])

# Extract year from release date
df['연도'] = df['개봉일'].dt.year

# Calculate sum of sales by distributor and year
sales_by_company_year = df.groupby(['배급사', '연도'], as_index=False).sum(numeric_only=True)

# Calculate sum of audience by distributor and year
audience_by_company_year = df.groupby(['배급사', '연도'], as_index=False).sum(numeric_only=True)

# Select top 10 distributors by sales each year
top_sales_companies_by_year = sales_by_company_year.groupby('연도').apply(lambda x: x.nlargest(10, '매출액')).reset_index(drop=True)
top_audience_companies_by_year = audience_by_company_year.groupby('연도').apply(lambda x: x.nlargest(10, '관객수')).reset_index(drop=True)

# Visualize sales (top 10 distributors)
fig_sales_top10 = px.bar(top_sales_companies_by_year, x='연도', y='매출액', color='배급사', title='연도별 상위 10개 배급사의 매출액')
fig_sales_top10.show()
fig_sales_top10.write_html('sales_top10.html')

# Visualize audience (top 10 distributors)
fig_audience_top10 = px.bar(top_audience_companies_by_year, x='연도', y='관객수', color='배급사', title='연도별 상위 10개 배급사의 관객수')
fig_audience_top10.show()
fig_audience_top10.write_html('audience_top10.html')


In [10]:
# 배급사별 가장 많은 매출을 올린 영화 찾기
top_movies = df.loc[df.groupby('배급사')['매출액'].idxmax()]

# 매출액을 기준으로 내림차순으로 정렬하고 상위 10개만 선택
top_movies = top_movies.sort_values(by='매출액', ascending=False).head(10)

# 시각화
fig = px.bar(top_movies, x='배급사', y='매출액', color='only_gener', title='배급사별 가장 많은 매출을 올린 영화의 장르 (상위 10개)')
fig.show()

# 장르별 가장 많은 매출을 기록한 영화 찾기
top_movies_by_genre = df.loc[df.groupby('only_gener')['매출액'].idxmax()]

# 매출액을 기준으로 내림차순으로 정렬하고 상위 10개만 선택
top_movies_by_genre = top_movies_by_genre.sort_values(by='매출액', ascending=False).head(10)

# 시각화
fig = px.bar(top_movies_by_genre, x='only_gener', y='매출액', color='배급사', title='장르별 가장 많은 매출을 기록한 배급사 (상위 10개)')
fig.show()

# 재호님 코드

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast
import itertools
import networkx as nx
import matplotlib.font_manager as fm
import plotly.express as px

data = pd.read_csv('../5.4_edit_maker.csv')


# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 집계
genre_actor_counts = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    
    for genre in genres:
        if genre not in genre_actor_counts:
            genre_actor_counts[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_counts[genre]:
                genre_actor_counts[genre][actor] = 0
            genre_actor_counts[genre][actor] += 1

# 데이터 프레임으로 변환
rows = []
for genre, actor_counts in genre_actor_counts.items():
    for actor, count in actor_counts.items():
        if count >= 3:  # 3회 이상 출연한 배우만 포함
            rows.append({'Genre': genre, 'Actor': actor, 'Count': count})

df_actor_counts = pd.DataFrame(rows)

# 데이터를 출연 횟수에 따라 내림차순 정렬
df_actor_counts.sort_values(by=['Genre', 'Count'], ascending=[True, True], inplace=True)

# 시각화
fig = px.bar(df_actor_counts, x='Genre', y='Count', color='Actor', title='장르별 배우 출연 횟수 (3회 이상 출연한 배우)',
             category_orders={"Genre": sorted(df_actor_counts['Genre'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [18]:
data = pd.read_csv('../5.4_edit_maker.csv')

# 매출액과 관객수 데이터 형태 처리
data['매출액'] = data['매출액'].str.replace(',', '').astype(float)
data['관객수'] = data['관객수'].str.replace(',', '').astype(int)

# 데이터 형태 확인 및 처리
def safe_eval(x):
    if isinstance(x, str):
        try:
            return eval(x)
        except:
            return []  # 오류 발생시 빈 리스트 반환
    return x if isinstance(x, list) else []

data['actor_main_name'] = data['actor_main_name'].apply(safe_eval)
data['actor_sub_name'] = data['actor_sub_name'].apply(safe_eval)

# 장르명 정규화 함수
def normalize_genre(genre):
    genre = genre.strip()  # 공백 제거
    genre = genre.replace(' ', '')  # 중간 공백 제거
    genre = genre.lower()  # 소문자로 통일
    return genre

# 장르별로 배우 출연 횟수 및 매출, 관객수 집계
genre_actor_stats = {}

for index, row in data.iterrows():
    genres = [normalize_genre(gen) for gen in row['gener'].split(',')]
    actors = set(row['actor_main_name'] + row['actor_sub_name'])  # 주연과 조연 합치기
    revenue = row['매출액']
    audience = row['관객수']
    
    for genre in genres:
        if genre not in genre_actor_stats:
            genre_actor_stats[genre] = {}
        
        for actor in actors:
            if actor not in genre_actor_stats[genre]:
                genre_actor_stats[genre][actor] = {'Count': 0, 'Total Revenue': 0, 'Total Audience': 0}
            genre_actor_stats[genre][actor]['Count'] += 1
            genre_actor_stats[genre][actor]['Total Revenue'] += revenue
            genre_actor_stats[genre][actor]['Total Audience'] += audience

# 데이터 프레임으로 변환
rows = []
for genre, actor_stats in genre_actor_stats.items():
    for actor, stats in actor_stats.items():
        if stats['Count'] >= 3:  # 3회 이상 출연한 배우만 포함
            rows.append({
                'Genre': genre, 
                'Actor': actor, 
                'Count': stats['Count'],
                'Total Revenue': stats['Total Revenue'],
                'Total Audience': stats['Total Audience']
            })

df_actor_stats = pd.DataFrame(rows)

# 데이터를 출연 횟수에 따라 내림차순 정렬
df_actor_stats.sort_values(by=['Genre', 'Count'], ascending=[True, False], inplace=True)

# 시각화
fig = px.bar(df_actor_stats, x='Genre', y='Count', color='Actor', title='장르별 배우 출연 횟수 (3회 이상 출연한 배우)',
             hover_data=['Total Revenue', 'Total Audience'],
             category_orders={"Genre": sorted(df_actor_stats['Genre'].unique(), reverse=True)})
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()